# Amazon - Baseline

### Pruebas con los distintos clasificadores para obtener los valores de transfer loss sin realizar adaptacion entre dominios

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *


#clasificadores
from utils.clasificacion import *

#otros
import os
import numpy as np
import pandas as pd

#variables para guardar los resultados
tipo = pruebas[0]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

In [ ]:
print tipo
print dataset_name
print dims
print data_path

## Pruebas con el dataset Amazon

In [ ]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
unlabeled = dataset_object.unlabeled
domains = dataset_object.domains

In [ ]:
i = 1

for tgt in domains:
    print "Entrenando dominio %d de %d" % (i, len(domains))
    X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "indomain_%s.pkl" % (tgt)
    model_path = os.path.join(models_path, dataset_name, model_name)

    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    
    print "Score: %.3f" % svc.score(X_ts, y_ts)
    i = i+1
    
print "\nTarea terminada."

In [ ]:
df = pd.DataFrame(columns=dataframe_columns)


i=0
# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 12" % (i+1)
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            
            #transfer error
            #entrenado en dominio src y probado en dominio tgt sin adaptar
            X_tr = np.asarray(labeled[src]['X_tr'].todense())
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "%s_%s_%s.pkl" % (tipo,src, tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            svc2 = load_best_score(model_path, X_tr, y_tr)
            t_error = 1-svc2.score(X_ts, y_ts)
            
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['Baseline',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            i+=1
    
print "Pruebas completadas."

In [ ]:
df

In [ ]:
new_scores_path = os.path.join(scores_path, tipo, dataset_name+".csv")

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."